# Deploy a serverless App

In this module we'll create a serverless application with Lambda than needs external libraries. We'll use [SAM (Serverless Application Model)](https://aws.amazon.com/serverless/sam/) to build and package this Lambda for us and deploy it with Amazon CloudFormation template

## Open a terminal

We need to interact with CLI and this will be hard from the notebook. Now click on the plus icon to open a new terminal

<img src="screenshots/navi.png" alt="nav_bar" />

<img src="screenshots/terminal.png" alt="terminal"/>


## Install SAM

First we need to install SAM to install our first serverless APP

execute the following in the terminal
```bash
curl -L https://github.com/aws/aws-sam-cli/releases/latest/download/aws-sam-cli-linux-x86_64.zip -o aws-sam-cli-linux-x86_64.zip
unzip aws-sam-cli-linux-x86_64.zip -d sam-installation
sudo ./sam-installation/install
which sam
sam --version
```

## Let's download an application

We'll clone a sample application from git

The sample app takes an architecture diagram as a png image from you and transforms it into a blog post as a PDF using Amazon Bedrock

In the terminal execute
```bash
git clone https://github.com/aws-samples/amazon-bedrock-claudev3-sonnet-blog-generation.git
```

## Add a build step to the application

Go to folder `amazon-bedrock-claudev3-sonnet-blog-generation/src` right click and choose 'create a new file' and create a new file named `requirements.txt`

Add the folloing line to the file and then press enter to add an empty new line

`fpdf2`

This is a library that is needed to create PDF files from the Lambda function

Now let's build our application

In the terminal execute
```bash
cd ./amazon-bedrock-claudev3-sonnet-blog-generation
sam build
```

You should see
```bash
Build Succeeded
```

Note that SAM downloaded the dependencies automatically. You can find them by inspecting the deployed Lambda code latter.

## Deploy the APP

### Option 1 guided experience
You can use SAM to guide you through deployment by specifying `-g`
In the terminal execute
```bash
sam deploy -g
```

Add stack name as `bedrock-app`, region as `us-west-2` and press enter on the rest to use default inputs.

### Option 2 automated
Or you can use command line paramters to automaticaly deploy the app
```bash
sam deploy \
  --stack-name bedrock-app \
  --region us-west-2 \
  --resolve-s3 \
  --capabilities CAPABILITY_IAM \
  --no-confirm-changeset
```


Once you get a confirmation message that app is deployed you can continue with testing

```bash
Successfully created/updated stack - bedrock in us-west-2
```

## Let's check the Lambda code

Going to the [AWS Lambda console](https://us-west-2.console.aws.amazon.com/lambda/home?region=us-west-2#/functions) and checking your newly created Lambda, you should see the library files automaticaly added to your Lambda

<img src="screenshots/lambda.png" alt="lambda" />

## Now let's try the app

Try to upload `sample-architecture.jpeg` image found in the repository `amazon-bedrock-claudev3-sonnet-blog-generation` to your [input bucket](https://us-west-2.console.aws.amazon.com/s3/home?region=us-west-2#). Then go and check the result in the output bucket
<img src="screenshots/buckets.png" alt="nav_bar" />

You should see an object like this
<img src="screenshots/output.png" alt="output" />


## Let's add more custom observability

We can add more CloudWatch metrics to our function to indicate that certain parts of the code were invoked or to monitor some outputs of our processes.

Let's go the `amazon-bedrock-claudev3-sonnet-blog-generation/template.yaml` file and add a policy to our Lambda

```yaml
- Version: '2012-10-17'
  Statement:
    - Effect: Allow
      Action: 
        - "cloudwatch:PutMetricData"
      Resource: "*"
```
***Note: if you find it hard to adjust yaml, you can use the prepared ```samples/template-with-permission.yaml``` instead***

Additionaly now we need to log something new from our function

Go inside the `amazon-bedrock-claudev3-sonnet-blog-generation/src` folder and open `app.py`

Right after this line
```python
# Parse the response body
response_body = json.loads(response.get('body').read())
print(response_body)
```

We can add the following lines
```python
cloudwatch = boto3.client('cloudwatch')

cloudwatch.put_metric_data(
    MetricData = [
        {
            'MetricName': 'InputTokens',
            'Unit': 'Count',
            'Value': response_body["usage"]["input_tokens"]
        },
        {
            'MetricName': 'OutputTokens',
            'Unit': 'Count',
            'Value': response_body["usage"]["output_tokens"]
        },
    ],
    Namespace='MyAppTokens'
)
```

***Note: if you find it hard to adjust python identiation, you can use the prepared ```samples/app-with-cloudwatch.py``` instead***

## Build, Deploy, Test

Now you need to prepat the following steps
- Build: we need to invoke `sam build` from the terminal to update the lambda code
- Deploy: We need to upload the changed lambda code and our changed template that now adds cloudwatch loging pemissions with `sam deploy`
- Finally we can reupload our sample image to input bucket, but now we need to observe additional metrics


## Check metrics in CLoudWatch metrics (takes 5 mins)

Now you can go check your new custom metric in the [Amazon Cloudwatch console](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#metricsV2?graph=~(view~'timeSeries~stacked~false~metrics~(~(~'MyAppTokens~'InputTokens)~(~'.~'OutputTokens))~region~'us-west-2~start~'-PT1H~end~'P0D)&query=~'*7bMyAppTokens*7d)

## Final thoughts

You can also add custom metrics to your AWS StepFunctions workflows to keep AWS Lambda code clean

<img src="screenshots/stepfunctions.png" alt="terminal"/>
